James Gardner, 2022

In [ ]:
from basic_benchmarking import *

### Replicating Fig 3 from Borhanian 2021 to test understanding, then try for CE only

In [ ]:
# for GW170817 and tf2_tidal

# select network
net, locs = network.Network(['aLIGO_H','aLIGO_L','aLIGO_V']), ['H', 'L', 'V']
# net, locs = network.Network(['CE1-40-CBO_C', 'CE1-40-CBO_S']), ['C', 'S']
# to-do: for CE only, the FIM is ill-conditioned, how to fix this?

# not stated, using one from GW170817 paper
# start with 100 sample points, then move up from there (e.g. to 1e4)
fmin, fmax, fnum = 30, 2048, 100
f = np.linspace(fmin, fmax, fnum)

wf_model_name = 'tf2_tidal'
wf_other_var_dic = None
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters for GW170817, reported median values (source-frame)
# using low-spin priors (Borhanian doesn't say which they used)
# https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.119.161101
# subsequent work (2019) has refined these values
base_measured_params = {
    'Mc':    1.188, # Msun
    'eta':   0.2485, # m1=1.48 Msun, m2=1.265 Msun, m2/m1=0.85, Mtot=2.74 Msun, eta=m1*m2/(m1+m2)**2
    'chi1z': 0,
    'chi2z': 0,
    'DL':    40, # MPc
    'tc':    0, # not quoted
    'phic':  0,
#     'iota':  PI/4, # 1000 random instances of these, measured: 55deg
#     'ra':    PI/4, # 1000 random instances of these
#     'dec':   PI/4, # 1000 random instances of these
    'psi':   PI/4, # not quoted, to-do: check effect
    'gmst0': 0, # not quoted but doesn't matter?
    'lam_t': 800, # combined dimensionless tidal deformability
    'delta_lam_t': 0 # not quoted, but approximating as zero because distributed around zero (check)
}

# assign with respect to which parameters to take derivatives, for the FIM, all 12 but not delta_lam_t (or gmst0)
deriv_symbs_string = 'Mc eta chi1z chi2z DL tc phic iota ra dec psi lam_t'

# assign which parameters to convert to log or cos versions for differentiation
conv_log = ('Mc', 'DL', 'lam_t')
conv_cos = ('iota', 'dec')

# choose whether to take Earth's rotation into account
use_rot = 0
# whether to calculate snr, errors, sky area for just the network and not the individual detectors
only_net = 1

# create lambdified derivatives for speed
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# starting with just 10 instances, scale up to 1000 later
num_instances = 1000 # 1000 is O(10 min) with numerical derivatives but is O(1 min) with symbolic derivatives
file_tag = f'{num_instances}instances_{net_label_styler(net.label)}'

In [ ]:
# angles are sampled to avoid clumping at poles
iota_ra_dec_randoms = np.transpose(injections.angle_sampler(num_instances, np.random.randint(100))[:-1])

def calculate_snr_errs_skyarea(iota_ra_dec):
    """given an array of iota, ra, dec; return an array of the integrate snr, Mc and DL errors, and 90% sky area"""
    iota, ra, dec = iota_ra_dec
    inj_params = dict(**base_measured_params, iota=iota, ra=ra, dec=dec)

    # copy network to avoid parallel operations conflicting, is this an issue when Pool() makes separate instances?
    net_copy = deepcopy(net)

    net_copy.set_net_vars(
        f=f, inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        conv_cos=conv_cos, conv_log=conv_log,
        use_rot=use_rot
    )

    basic_network_benchmarking(net_copy, numerical_over_symbolic_derivs=False, only_net=only_net)

    if net_copy.wc_fisher: # i.e. net.cond_num < 1e15:
        # net_copy is automatically deleted once out of scope
        return net_copy.snr, net_copy.errs['log_Mc'], net_copy.errs['log_DL'], net_copy.errs['sky_area_90']
    else:
        # try again with different random values
        # to-do: add counter to quantify rate of occurance; seems to be v common with CE only, why?
        #print(f'{net_copy.cond_num:.3g} is ill-conditioned, try again')
        #return calculate_snr_errs_skyarea(np.transpose(injections.angle_sampler(1, np.random.randint(100))[:-1]))
        # alternative to guarantee halting 
        return np.full(4, np.nan)

# array to store integrated SNR, 1sigma error estimates (for Mc and DL), and 90% credible sky area
# must be careful with parallelising that the network is not used simultaneously
# keeping one cpu free to use laptop
results_snr_errs_skyarea = np.array(p_umap(calculate_snr_errs_skyarea, iota_ra_dec_randoms, num_cpus=3))

# save results
np.save(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy', results_snr_errs_skyarea)

In [ ]:
# load results
results_snr_errs_skyarea = np.load(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy')

# filter out NaNs
results_snr_errs_skyarea = without_rows_w_nan(results_snr_errs_skyarea)

if len(results_snr_errs_skyarea) == 0:
    print('All values are NaN, FIM is ill-conditioned.')
else:
    print('Some values are not NaN.')

In [ ]:
# measured values
# given Appendix E: Asymmetric Systematic Uncertainties, estimating the st.dev. from asymm errs depends on your model
# in the Gaussian case, just average the upper and lower errors that correspond to 68% of the data
# given a 90% credibility interval (x-xlower,x+xupper), the st.dev. is approximated by 1/1.64*1/2(xlower+xupper)
meas_snr = 32.4
# too high eyeballing wrt Borhanian 2021?
meas_Mc_rel_err = 1/1.64*1/2*(0.004+0.002)/1.188 # 1.188+0.004-0.002, range for 90% credibility
# too low eyeballing wrt Borhanian 2021?
meas_LD_rel_err = 1/1.64*1/2*(8+14)/40 # 40+8−14
meas_sky_area = 28
meas_vals = meas_snr, meas_Mc_rel_err, meas_LD_rel_err, meas_sky_area

# plotting
# comparing symmetricised relative errors rather than 90% credible bounds
plt.rcParams.update({'font.size': 14})
fig, axs = plt.subplots(1, 4, figsize=(18, 2), sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0})

for i, ax in enumerate(axs):
    data = results_snr_errs_skyarea[:,i]
    ax.hist(data, histtype='step', facecolor='b', label='tf2_tidal',
            bins=np.geomspace(data.min(), data.max(), 50))
    ax.axvline(meas_vals[i], color='grey')
    ax.set_xscale('log')
    # NB: when there's no major tick, the axis has no reference value
    ax.xaxis.set_minor_locator(plt.LogLocator(base=10.0, subs=0.1*np.arange(1, 10), numticks=10))
    ax.xaxis.set_minor_formatter(plt.NullFormatter())

snr_xlim = axs[0].get_xlim()
for snr_threshold in SNR_THRESHOLD_LO, SNR_THRESHOLD_HI:
    efficiency = np.mean(results_snr_errs_skyarea[:,0] > snr_threshold)
    print(f"efficiency of network is {efficiency:.1%} wrt SNR threshold {snr_threshold}")
axs[0].axvspan(snr_xlim[0], SNR_THRESHOLD_LO, alpha=0.5, color='lightgrey')
axs[0].axvspan(snr_xlim[0], SNR_THRESHOLD_HI, alpha=0.25, color='lightgrey')
axs[0].set_xlim(snr_xlim)

axs[0].set_ylabel('GW170817\ncount')
axs[0].set_xlabel(r'integrated SNR, $\rho$')
# error in log(X) is the fractional error in X (i.e. (error in X)/X) by chain rule 
axs[1].set_xlabel(r'chirp mass, $\Delta\mathcal{M}/\mathcal{M}$')
axs[2].set_xlabel(r'luminosity distance, $\Delta D_L/D_L$')
axs[3].set_xlabel(r'sky area, $\Omega$ / $\mathrm{deg}^2$')
axs[0].legend()

fig.align_labels()
fig.savefig(f'plots/GW170817_histograms_{file_tag}.pdf', bbox_inches='tight')
plt.show(fig)
plt.close(fig)
# results for HLV disagree with Borhanian2021: sharpness of first twos' fall-offs, centre of snr distribution, measured Mc and DL, extent of sky area curve
# however, with a difference 1000 instances, the results change --- so maybe just a sampling issue